In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# prefix path
prefix = '/content/drive/MyDrive/데이터캠프/DCC/cv/task1/mini'

In [3]:
from keras.applications.xception import Xception
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import time
import datetime
import cv2

# 데이터 다운로드

> CIFAR-10은 32 x 32 크기의 60000개의 이미지로 이루어져 있습니다.
10개의 클래스로 분류(airplane, bird, cat ...) 되며 각각의 클래스는 6000개의 이미지로 이루어져 있습니다.
또, 5000개는 학습 데이터, 1000개는 테스트 데이터입니다
(총, 50000개의 학습 데이터, 10000개의 테스트 데이터).



In [4]:
#데이터 셋
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [5]:
# dataset shape
print(train_images.shape, test_images.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


# 데이터 전처리


*   x는 255로 나누어 줘서 0에서 1사이로 변환시켜줌
*   y는 원 핫 인코딩을 적용함



In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(train_images, 
                                                  train_labels, 
                                                  test_size = 0.2, 
                                                  random_state=100)

In [7]:
X_train.shape, X_val.shape

((40000, 32, 32, 3), (10000, 32, 32, 3))

In [8]:
# the number of classes
num_class = len(np.unique(Y_val))
num_class

10

*https://github.com/Deep-as-hell-team/Fashion-MNIST-/blob/master/Xception.ipynb 의 코드 참고*

In [9]:
def resize_Xcept(img):
  img = img.reshape(32,32,3).astype('float32') / 255.0  
  img = cv2.resize(img, (84,84)) #리사이징  
  return img

In [11]:
X_train2 = [resize_Xcept(img) for img in X_train]
X_train2 = np.asarray(X_train2)

X_val2 = [resize_Xcept(img) for img in X_val]
X_val2 = np.asarray(X_val2)

In [12]:
test_images2 = [resize_Xcept(img) for img in test_images]
test_images2 = np.asarray(test_images2)

In [13]:
# make one-hot label
Y_train = keras.utils.to_categorical(Y_train)
Y_val = keras.utils.to_categorical(Y_val)
test_labels = keras.utils.to_categorical(test_labels)

# No Argumentation 모델
---
Xception: 구글이 발표한 cnn모델




In [ ]:
base = Xception(weights = None, include_top = False, input_shape=(84,84,3))

In [ ]:
model = keras.models.Sequential()
model.add(base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(num_class, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 21,045,810
Trainable params: 20,991,282
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
start = time.time()
# 코랩 램이 더 이상의 batch_size를 크게 하면 감당못함
train_model = model.fit(X_train2, Y_train,
                        batch_size=64,                    
                        epochs = 4,
                        validation_data =(X_val2, Y_val))

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Epoch 1/4
625/625 [==============================] - 284s 398ms/step - loss: 1.4571 - accuracy: 0.4891 - val_loss: 117.2454 - val_accuracy: 0.2565
Epoch 2/4
625/625 [==============================] - 246s 394ms/step - loss: 1.0522 - accuracy: 0.6284 - val_loss: 2.7838 - val_accuracy: 0.6255
Epoch 3/4
625/625 [==============================] - 252s 404ms/step - loss: 0.8463 - accuracy: 0.7054 - val_loss: 21.7740 - val_accuracy: 0.4553
Epoch 4/4
625/625 [==============================] - 246s 393ms/step - loss: 0.7903 - accuracy: 0.7270 - val_loss: 0.8556 - val_accuracy: 0.6952
0:17:30


In [ ]:
model.evaluate(test_images2, test_labels)

313/313 [==============================] - 17s 54ms/step - loss: 0.8710 - accuracy: 0.6876


[0.8709840774536133, 0.6876000165939331]

# Data Argumentation

In [14]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,      
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,        
        fill_mode='nearest'
        )

In [25]:
train_argu = datagen.flow(X_train2, Y_train, batch_size=128)

In [19]:
base = Xception(weights = None, include_top = False, input_shape=(84,84,3))

In [20]:
model = keras.models.Sequential()
model.add(base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(num_class, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 21,045,810
Trainable params: 20,991,282
Non-trainable params: 54,528
_________________________________________________________________


In [28]:
start = time.time()
# 코랩 램이 더 이상의 batch_size를 크게 하면 감당못함
train_model = model.fit(train_argu,
                        batch_size=64,                    
                        epochs = 4,
                        validation_data =(X_val2, Y_val))

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Epoch 1/4
313/313 [==============================] - 236s 754ms/step - loss: 1.2936 - accuracy: 0.5356 - val_loss: 1.3733 - val_accuracy: 0.5316
Epoch 2/4
313/313 [==============================] - 241s 769ms/step - loss: 1.1557 - accuracy: 0.5834 - val_loss: 1.1121 - val_accuracy: 0.6271
Epoch 3/4
313/313 [==============================] - 235s 751ms/step - loss: 1.0566 - accuracy: 0.6259 - val_loss: 2.0799 - val_accuracy: 0.5897
Epoch 4/4
313/313 [==============================] - 236s 752ms/step - loss: 0.9785 - accuracy: 0.6581 - val_loss: 1.1284 - val_accuracy: 0.6178
0:15:48


In [29]:
model.evaluate(test_images2, test_labels)

313/313 [==============================] - 17s 54ms/step - loss: 1.1427 - accuracy: 0.6059


[1.1427497863769531, 0.60589998960495]

---
#### 혹시 이미지를 너무 많이 조정해서 성능이 떨어지는 걸까?

In [ ]:
# 상하좌우 이동만으로 제한
datagen = ImageDataGenerator(        
        width_shift_range=0.2,
        height_shift_range=0.2        
        )

In [ ]:
train_argu = datagen.flow(X_train2, Y_train, batch_size=256)

In [ ]:
base = Xception(weights = None, include_top = False, input_shape=(84,84,3))

In [ ]:
model = keras.models.Sequential()
model.add(base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(num_class, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
Total params: 21,045,810
Trainable params: 20,991,282
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
start = time.time()
# 코랩 램이 더 이상의 batch_size를 크게 하면 감당못함
train_model = model.fit(train_argu,
                        batch_size=64,                    
                        epochs = 4,
                        validation_data =(X_val2, Y_val))

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

Epoch 1/4
313/313 [==============================] - 236s 754ms/step - loss: 1.2936 - accuracy: 0.5356 - val_loss: 1.3733 - val_accuracy: 0.5316
Epoch 2/4
313/313 [==============================] - 241s 769ms/step - loss: 1.1557 - accuracy: 0.5834 - val_loss: 1.1121 - val_accuracy: 0.6271
Epoch 3/4
253/313 [=======================>......] - ETA: 42s - loss: 1.0593 - accuracy: 0.6256

In [ ]:
model.evaluate(test_images2, test_labels)

313/313 [==============================] - 17s 52ms/step - loss: 1.4282 - accuracy: 0.4806


[1.4282002449035645, 0.4805999994277954]